# Timeseries 2 - Statistical Modeling (~15 minutes)

## Layout:

- (1) - Exponential Filtering (5 minutes)
- (2) - ARIMA (5 minutes)
- (3) - Prophet (5 minutes)

In [ ]:
### if you want to run on your own computer => upgrade required package
# ! pip install matplotlib --upgrade
# ! pip install pandas --upgrade
# ! pip install seaborn --upgrade
# ! pip install plotly --upgrade
# ! pip install pystan --upgrade
# ! pip install statsmodels
# ! pip install prophet --upgrade

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Loading data with read_csv:

We do two specific things while loading:

- `usecols`: We only consider the datetime and the count series
- `parse_dates` : We parse the datetime serie as dates

NB: [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html?highlight=read_csv#pandas.read_csv) has a TON of options, be sure to check them

In [ ]:
#lets load the data and only consider the count as a serie.
df = pd.read_csv("https://raw.githubusercontent.com/vguigue/TimeSeries/main/data/train.csv",parse_dates=["datetime"],usecols=['datetime','count'])
df["minutes"] = df.datetime.dt.minute
df["hour"]  = df.datetime.dt.hour
df["day"]  = df.datetime.dt.day
df["month"]  = df.datetime.dt.month
df["year"]  = df.datetime.dt.year
df["weekday"]  = df.datetime.dt.day_of_week


time_indexed = df
time_indexed = time_indexed.set_index("datetime")
time_indexed.head()


# Simple forecasts : can we predict the two future days ?

- Step 1: splitting the data before/after a given date

- Step 2: use pandas embedded models

In [ ]:
month_data_train = time_indexed.loc["06-01-2011":"06-17-2011"].copy()
month_data_test = time_indexed.loc["06-18-2011":"06-19-2011"].copy()

month_data_train["count"].plot(figsize=(25,12))
month_data_test["count"].plot(figsize=(25,12))

In [ ]:
month_data_train

Let's start with a naive hypothesis: "tomorrow will be the same as today". However, instead of a model like $\hat{y}_{t} = y_{t-1}$ (which is actually a great baseline for any time series prediction problems and sometimes is impossible to beat), we will assume that the future value of our variable depends on the average of its $k$ previous values. Therefore, we will use the **moving average**.

$\hat{y}_{t} = \frac{1}{k} \displaystyle\sum^{k}_{n=1} y_{t-n}$

Unfortunately, we cannot make predictions far in the future - in order to get the value for the next step, we need the previous values to be actually observed. But moving average has another use case - smoothing the original time series to identify trends. Pandas has an implementation available with [`DataFrame.rolling(window).mean()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html). The wider the window, the smoother the trend. In the case of very noisy data, which is often encountered in finance, this procedure can help detect common patterns.



In [ ]:
month_data_train["ma_2h"] = month_data_train["count"].rolling(window=2).mean()
month_data_train["ma_6h"] = month_data_train["count"].rolling(window=6).mean()

month_data_train[["count","ma_2h","ma_6h"]].plot(figsize=(25,12))



### => as it can be seen on the graph, it's hard to shift the mean forward to predict accurately.

## Exponential smoothing

Now, let's see what happens if, instead of weighting the last $k$ values of the time series, we start weighting all available observations while exponentially decreasing the weights as we move further back in time. There exists a formula for **[exponential smoothing](https://en.wikipedia.org/wiki/Exponential_smoothing)** that will help us with this:

$$\hat{y}_{t} = \alpha \cdot y_t + (1-\alpha) \cdot \hat y_{t-1} $$

Here the model value is a weighted average between the current true value and the previous model values. The $\alpha$ weight is called a smoothing factor. It defines how quickly we will "forget" the last available true observation. The smaller $\alpha$ is, the more influence the previous observations have and the smoother the series is.

Exponentiality is hidden in the recursiveness of the function – we multiply by $(1-\alpha)$ each time, which already contains a multiplication by $(1-\alpha)$ of previous model values

This is also [built in pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html)

In [ ]:

month_data_train["ewm_05"] = month_data_train["count"].ewm(alpha=0.5,adjust=False).mean()
month_data_train["ewm_01"] = month_data_train["count"].ewm(alpha=0.1,adjust=False).mean()

month_data_train[["count","ewm_05","ewm_01"]].plot(figsize=(25,12))

# Statsmodels -- Straightforward series analysis & Forecasting

## Statsmodels

### First, you can do the same things as with pandas: like exp. smoothing:

#### (a) Setting a DatetimeIndex Frequency
Note that our DatetimeIndex does not have a frequency. In order to build a Holt-Winters smoothing model, statsmodels needs to know the frequency of the data (whether it's daily, monthly etc.). Since observations occur each hour, we'll use H.<br>A full list of time series offset aliases can be found <a href='http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases'>here</a>.

In [ ]:
month_data_train.index.freq = "H"
month_data_test.index.freq = "H"
month_data_train.index



Simple exp. smoothing yields the same values:

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

#For some reason, when optimized=False is passed into .fit()
#the statsmodels SimpleExpSmoothing function shifts fitted values down one row.
#We fix this by adding .shift(-1) after .fittedvalues

month_data_train['sm_ewm_05']  = SimpleExpSmoothing(month_data_train["count"]).fit(smoothing_level=0.5,optimized=False).fittedvalues.shift(-1)
month_data_train['sm_ewm_01']  = SimpleExpSmoothing(month_data_train["count"]).fit(smoothing_level=0.1,optimized=False).fittedvalues.shift(-1)

month_data_train[["count","sm_ewm_05","sm_ewm_01"]].plot(figsize=(25,12))

month_data_train[["ewm_05","sm_ewm_05","ewm_01","sm_ewm_01"]].head()

## Forecasting with `.forecast`

In [ ]:
model = SimpleExpSmoothing(month_data_train["count"]).fit(smoothing_level=0.5,optimized=False)

month_data_test["ewm_05"] = model.forecast(48) # we forecast on 2 days

month_data_test[["count","ewm_05"]].plot(figsize=(25,12))


### Holt-Winters Methods
In the previous cells  we applied <em>Simple Exponential Smoothing</em> using just one smoothing factor $\alpha$ (alpha). This failed to account for other contributing factors like trend and seasonality.

In this section we'll look at <em>Double</em> and <em>Triple Exponential Smoothing</em> with the <a href='https://otexts.com/fpp2/holt-winters.html'>Holt-Winters Methods</a>.

In <strong>Double Exponential Smoothing</strong> (aka Holt's Method) we introduce a new smoothing factor $\beta$ (beta) that addresses trend:

\begin{split}l_t &= (1 - \alpha) l_{t-1} + \alpha x_t, & \text{    level}\\
b_t &= (1-\beta)b_{t-1} + \beta(l_t-l_{t-1}) & \text{    trend}\\
y_t &= l_t + b_t & \text{    fitted model}\\
\hat y_{t+h} &= l_t + hb_t & \text{    forecasting model (} h = \text{# periods into the future)}\end{split}

Because we haven't yet considered seasonal fluctuations, the forecasting model is simply a straight sloped line extending from the most recent data point. We'll see an example of this in upcoming lectures.

With <strong>Triple Exponential Smoothing</strong> (aka the Holt-Winters Method) we introduce a smoothing factor $\gamma$ (gamma) that addresses seasonality:

\begin{split}l_t &= (1 - \alpha) l_{t-1} + \alpha x_t, & \text{    level}\\
b_t &= (1-\beta)b_{t-1} + \beta(l_t-l_{t-1}) & \text{    trend}\\
c_t &= (1-\gamma)c_{t-L} + \gamma(x_t-l_{t-1}-b_{t-1}) & \text{    seasonal}\\
y_t &= (l_t + b_t) c_t & \text{    fitted model}\\
\hat y_{t+m} &= (l_t + mb_t)c_{t-L+1+(m-1)modL} & \text{    forecasting model (} m = \text{# periods into the future)}\end{split}

Here $L$ represents the number of divisions per cycle. In our case looking at monthly data that displays a repeating pattern each year, we would use $L=12$.

In general, higher values for $\alpha$, $\beta$ and $\gamma$ (values closer to 1), place more emphasis on recent data.

<h3>Related Functions:</h3>
<tt><strong><a href='https://www.statsmodels.org/stable/generated/statsmodels.tsa.holtwinters.SimpleExpSmoothing.html'>statsmodels.tsa.holtwinters.SimpleExpSmoothing</a></strong><font color=black>(endog)</font>&nbsp;&nbsp;&nbsp;&nbsp;
Simple Exponential Smoothing<br>
<strong><a href='https://www.statsmodels.org/stable/generated/statsmodels.tsa.holtwinters.ExponentialSmoothing.html'>statsmodels.tsa.holtwinters.ExponentialSmoothing</a></strong><font color=black>(endog)</font>&nbsp;&nbsp;
    Holt-Winters Exponential Smoothing</tt>
    
<h3>For Further Reading:</h3>
<tt>
<strong>
<a href='https://www.itl.nist.gov/div898/handbook/pmc/section4/pmc43.htm'>NIST/SEMATECH e-Handbook of Statistical Methods</a></strong>&nbsp;&nbsp;<font color=black>What is Exponential Smoothing?</font></tt>

___
## Double Exponential Smoothing
Where Simple Exponential Smoothing employs just one smoothing factor $\alpha$ (alpha), Double Exponential Smoothing adds a second smoothing factor $\beta$ (beta) that addresses trends in the data. Like the alpha factor, values for the beta factor fall between zero and one ($0<\beta≤1$). The benefit here is that the model can anticipate future increases or decreases where the level model would only work from recent calculations.

We can also address different types of change (growth/decay) in the trend. If a time series displays a straight-line sloped trend, you would use an <strong>additive</strong> adjustment. If the time series displays an exponential (curved) trend, you would use a <strong>multiplicative</strong> adjustment.

As we move toward forecasting, it's worth noting that both additive and multiplicative adjustments may become exaggerated over time, and require <em>damping</em> that reduces the size of the trend over future periods until it reaches a flat line.

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

#additive
month_data_train['doubleEs_add'] =ExponentialSmoothing(month_data_train["count"], trend='add').fit().fittedvalues.shift(-1)

#multiplicative
month_data_train['doubleEs_mul'] =ExponentialSmoothing(month_data_train["count"], trend='mul').fit().fittedvalues.shift(-1)

month_data_train[["count","doubleEs_add","doubleEs_mul"]].plot(figsize=(25,12))


In [ ]:
model2 = ExponentialSmoothing(month_data_train["count"], trend='add').fit()
model2mul = ExponentialSmoothing(month_data_train["count"], trend='mul').fit()

month_data_test["doubleEs_add"] = model2.forecast(48) # we forecast on 2 days
month_data_test["doubleEs_mul"] = model2mul.forecast(48) # we forecast on 2 days

month_data_test[["count","doubleEs_add","doubleEs_mul"]].plot(figsize=(25,12))

## Triple Exponential Smoothing
Triple Exponential Smoothing, the method most closely associated with Holt-Winters, adds support for both trends and seasonality in the data.


In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

month_data_train['tripleEs_add'] =ExponentialSmoothing(month_data_train["count"], trend='add',seasonal='add',seasonal_periods=24).fit().fittedvalues.shift(-1)

month_data_train[["count","doubleEs_add","doubleEs_mul","tripleEs_add"]].plot(figsize=(25,12))


## Forecast with triple exponential smoothing

In [ ]:
model3 = ExponentialSmoothing(month_data_train["count"], trend='mul',seasonal='add',seasonal_periods=24).fit()

month_data_test["tripleEs_add"] = model3.forecast(48)

month_data_test[["count","tripleEs_add"]].plot(figsize=(25,12))

## There are many models to model time series as "moving averages":

### [Be sure to check statsmodel docs](https://www.statsmodels.org/stable/index.html)

# Econometric approach

## Getting rid of non-stationarity and building SARIMA

Let's build an ARIMA model.

Here is the code to render plots.

In [ ]:
import seaborn as sns                            # more plots
sns.set()

from dateutil.relativedelta import relativedelta # working with dates with style
from scipy.optimize import minimize              # for function minimization

import statsmodels.formula.api as smf            # statistics and econometrics
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

from itertools import product                    # some useful functions
from tqdm import tqdm_notebook

import warnings                                  # `do not disturbe` mode
warnings.filterwarnings('ignore')

In [ ]:
def tsplot(y, lags=None, figsize=(12, 7), style='bmh'):
    """
        Plot time series, its ACF and PACF, calculate Dickey–Fuller test

        y - timeseries
        lags - how many lags to include in ACF, PACF calculation
    """
    if not isinstance(y, pd.Series):
        y = pd.Series(y)

    with plt.style.context(style):
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))

        y.plot(ax=ts_ax)
        p_value = sm.tsa.stattools.adfuller(y)[1]
        ts_ax.set_title('Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}'.format(p_value))
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()

In [ ]:
tsplot(month_data_train["count"], lags=60)

_this outlier on partial autocorrelation plot looks like a statsmodels bug, partial autocorrelation shall be <= 1 like any correlation._

Surprisingly, the initial series are stationary; the Dickey-Fuller test rejected the null hypothesis that a unit root is present. Actually, we can see this on the plot itself – we do not have a visible trend, so the mean is constant and the variance is pretty much stable. The only thing left is seasonality, which we have to deal with prior to modeling. To do so, let's take the "seasonal difference", which means a simple subtraction of the series from itself with a lag that equals the seasonal period.

In [ ]:
data_diff = month_data_train["count"] - month_data_train["count"].shift(24)
tsplot(data_diff[24:], lags=60)

It is now much better with the visible seasonality gone. However, the autocorrelation function still has too many significant lags. To remove them, we'll take first differences, subtracting the series from itself with lag 1.

In [ ]:
data_diff = data_diff - data_diff.shift(1)
tsplot(data_diff[24+1:], lags=60)

Perfect! Our series now looks like something undescribable, oscillating around zero. The Dickey-Fuller test indicates that it is stationary, and the number of significant peaks in ACF has dropped. We can finally start modeling!

## ARIMA-family Crash-Course

We will explain this model by building up letter by letter. $SARIMA(p, d, q)(P, D, Q, s)$, Seasonal Autoregression Moving Average model:

- $AR(p)$ - autoregression model i.e. regression of the time series onto itself. The basic assumption is that the current series values depend on its previous values with some lag (or several lags). The maximum lag in the model is referred to as $p$. To determine the initial $p$, you need to look at the PACF plot and find the biggest significant lag after which **most** other lags become insignificant.
- $MA(q)$ - moving average model. Without going into too much detail, this models the error of the time series, again with the assumption that the current error depends on the previous with some lag, which is referred to as $q$. The initial value can be found on the ACF plot with the same logic as before.

Let's combine our first 4 letters:

$AR(p) + MA(q) = ARMA(p, q)$

What we have here is the Autoregressive–moving-average model! If the series is stationary, it can be approximated with these 4 letters. Let's continue.

- $I(d)$ - order of integration. This is simply the number of nonseasonal differences needed to make the series stationary. In our case, it's just 1 because we used first differences.

Adding this letter to the four gives us the $ARIMA$ model which can handle non-stationary data with the help of nonseasonal differences. Great, one more letter to go!

- $S(s)$ - this is responsible for seasonality and equals the season period length of the series

With this, we have three parameters: $(P, D, Q)$

- $P$ - order of autoregression for the seasonal component of the model, which can be derived from PACF. But you need to look at the number of significant lags, which are the multiples of the season period length. For example, if the period equals 24 and we see the 24-th and 48-th lags are significant in the PACF, that means the initial $P$ should be 2.

- $Q$ - similar logic using the ACF plot instead.

- $D$ - order of seasonal integration. This can be equal to 1 or 0, depending on whether seasonal differeces were applied or not.

Now that we know how to set the initial parameters, let's have a look at the final plot once again and set the parameters:

In [ ]:
tsplot(data_diff[24+1:], lags=60)

- $p$ is most probably 4 since it is the last significant lag on the PACF, after which, most others are not significant.
- $d$ equals 1 because we had first differences
- $q$ should be somewhere around 4 as well as seen on the ACF
- $P$ might be 2, since 24-th and 48-th lags are somewhat significant on the PACF
- $D$ again equals 1 because we performed seasonal differentiation
- $Q$ is probably 1. The 24-th lag on ACF is significant while the 48-th is not.

Let's test various models and see which one is better.

In [ ]:
# grid searching => range can be reduced to save time in the practical session
# setting initial values and some bounds for them
ps = range(3, 5) # (2,5)
d=1
qs = range(3, 5) # (2,5)
Ps = range(1, 2) # (0,2)
D=1
Qs = range(1, 2) # (0,2)
s = 24 # season length is still 24

# creating list with all the possible combinations of parameters
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [ ]:
def optimizeSARIMA(parameters_list, d, D, s):
    """
        Return dataframe with parameters and corresponding AIC

        parameters_list - list with (p, q, P, Q) tuples
        d - integration order in ARIMA model
        D - seasonal integration order
        s - length of season
    """

    results = []
    best_aic = float("inf")

    for param in tqdm_notebook(parameters_list):
        # we need try-except because on some combinations model fails to converge
        try:
            model=sm.tsa.statespace.SARIMAX(month_data_train["count"], order=(param[0], d, param[1]),
                                            seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
        except:
            continue
        aic = model.aic
        # saving best model, AIC and parameters
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        results.append([param, model.aic])

    result_table = pd.DataFrame(results)
    result_table.columns = ['parameters', 'aic']
    # sorting in ascending order, the lower AIC is - the better
    result_table = result_table.sort_values(by='aic', ascending=True).reset_index(drop=True)

    return result_table

In [ ]:
%%time
#best are supposed to be : (2, 4, 1, 1)
result_table = optimizeSARIMA(parameters_list, d, D, s)

In [ ]:
result_table.head()

In [ ]:
# set the parameters that give the lowest AIC
# p, q, P, Q = result_table.parameters[0] # to save computation time
p, q, P, Q = (2, 4, 1, 1)

best_model=sm.tsa.statespace.SARIMAX(month_data_train["count"], order=(p, d, q),
                                        seasonal_order=(P, D, Q, s)).fit(disp=-1)
print(best_model.summary())

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def MSE(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

In [ ]:
def plotSARIMA(series, model, n_steps):
    """
        Plots model vs predicted values

        series - dataset with timeseries
        model - fitted SARIMA model
        n_steps - number of steps to predict in the future

    """
    # adding model values
    data = pd.DataFrame(data=series, columns=['actual'])
    data['arima_model'] = model.fittedvalues
    # making a shift on s+d steps, because these values were unobserved by the model
    # due to the differentiating
    data['arima_model'][:s+d] = np.NaN

    # forecasting on n_steps forward
    forecast = model.predict(start = data.shape[0], end = data.shape[0]+n_steps)
    forecast = data.arima_model.append(forecast)
    # calculate error, again having shifted on s+d steps from the beginning
    error = mean_absolute_percentage_error(data['actual'][s+d:], data['arima_model'][s+d:])

    plt.figure(figsize=(15, 7))
    plt.title("Mean Absolute Percentage Error: {0:.2f}%".format(error))
    plt.plot(forecast, color='r', label="model")
    plt.axvspan(data.index[-1], forecast.index[-1], alpha=0.5, color='lightgrey')
    plt.plot(data.actual, label="actual")
    plt.legend()
    plt.grid(True);

In [ ]:
plotSARIMA(month_data_train["count"], best_model, 30)

In the end, we got very adequate predictions. Our model was wrong by 4.01% on average, which is very, very good. However, the overall costs of preparing data, making the series stationary, and selecting parameters might not be worth this accuracy.

# Linear (and not quite) models for time series

Often, in my job, I have to build models with [*fast, good, cheap*](http://fastgood.cheap) as my only guiding principle. That means that some of these models will never be considered "production ready" as they demand too much time for data preparation (as in SARIMA) or require frequent re-training on new data (again, SARIMA) or are difficult to tune (good example - SARIMA). Therefore, it's very often much easier to select a few features from the existing time series and build a simple linear regression model or, say, a random forest. It is good and cheap.

This approach is not backed by theory and breaks several assumptions (e.g. Gauss-Markov theorem, especially for errors being uncorrelated), but it is very useful in practice and is often used in machine learning competitions.


# (3)  Prophet

> Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.


- **Accurate and fast** : Prophet is used in many applications across Facebook for producing reliable forecasts for planning and goal setting. We’ve found it to perform better than any other approach in the majority of cases. We fit models in Stan so that you get forecasts in just a few seconds.

- **Fully automatic** : Get a reasonable forecast on messy data with no manual effort. Prophet is robust to outliers, missing data, and dramatic changes in your time series.

- **Tunable forecasts** : The Prophet procedure includes many possibilities for users to tweak and adjust forecasts. You can use human-interpretable parameters to improve your forecast by adding your domain knowledge.

- **Available in R or Python** : We’ve implemented the Prophet procedure in R and Python, but they share the same underlying Stan code for fitting. Use whatever language you’re comfortable with to get forecasts.


**You should really read the paper for Prophet! It is relatively straightforward and has a lot of insight on their techniques on how Prophet works internally!**

Link to paper: https://peerj.com/preprints/3190.pdf

In [ ]:
! pip install prophet

In [ ]:
from prophet import Prophet

In [ ]:
month_data_train.head()

### Formatting data for Prophet:

> The input to Prophet is always a dataframe with two columns: ds and y. The ds (datestamp) column should be of a format expected by Pandas, ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp. The y column must be numeric, and represents the measurement we wish to forecast.

In [ ]:
prophet_data_train = month_data_train["count"].reset_index().rename({'datetime':'ds', 'count':'y'}, axis='columns').copy()
prophet_data_train.head()

In [ ]:
prophet_data_test = month_data_test["count"].reset_index().rename({'datetime':'ds', 'count':'y'}, axis='columns').copy()
prophet_data_test.head()

### Using Prophet

Prophet follows the sklearn model API. We create an instance of the Prophet class and then call its fit and predict methods.

**NOTE: Prophet by default is for daily data. You need to pass a frequency for sub-daily or monthly data. Info: https://facebook.github.io/prophet/docs/non-daily_data.html**

In [ ]:
mod = Prophet()
mod.fit(prophet_data_train)
future = mod.make_future_dataframe(periods=48, freq='H')
forecast = mod.predict(future)

## Let's see how the forecast performed

In [ ]:
predicted_days = forecast[-48:].set_index("ds").sort_index().copy()
predicted_days["ground_truth"] = month_data_test.sort_index()["count"].values

In [ ]:
predicted_days.head()

### plot the prediction:

In [ ]:
predicted_days[["yhat","ground_truth"]].reset_index(drop=True).plot(figsize=(25,12)) # for some reason, the index messes things up
# predicted_days["ground_truth"].reset_index(drop=True).plot(figsize=(25,12))

### => With prophet, you can easily see learnt componants.

In [ ]:
mod.plot_components(forecast)

## [If you have time left, have a look at prophet docs !](https://facebook.github.io/prophet/docs/quick_start.html#python-api)

#### => Especially, try to add a componant yourself like the holidays